In [1]:
reset()
%display latex
Partitions.options.latex="list"
from sage.misc.cachefunc import cached_method, cached_function
from sage.structure.unique_representation import UniqueRepresentation
from itertools import chain, combinations
from collections import deque

load("partition_4d.sage")



KT_vars = ['x1','x2','x3','x4','y','u1','u2','u3','u4','w']
KT_ring = LaurentPolynomialRing(QQ, KT_vars)


coho_vars = ['t1','t2','t3','t4','m']
coho_ring = LaurentPolynomialRing(QQ, coho_vars)


x1,x2,x3,x4,y,u1,u2,u3,u4,w=KT_ring.gens()
t1,t2,t3,t4,m= coho_ring.gens()
b=var('b')

pP4ds = Partitions4d()

def character(p,w=(x1,x2,x3,x4)):
    Nx1, Nx2, Nx3, Nx4 = p._Nx1, p._Nx2, p._Nx3, p._Nx4
    return substitute(sum(x1^(i) * x2^(j) * x3^(k) * x4^(l)
           for (i,j,k,l) in p.boxes(Nx1=Nx1, Nx2=Nx2, Nx3=Nx3, Nx4=Nx4)),w)

def dual(f):
    if f in ZZ:
        return f
    R = KT_ring
    g= R.zero()
    for exp, coeff in f.dict().items():
        g+= R({exp.emul(-1): coeff})
    return g


def specialize(f):
    if f in ZZ:
        return f
    return f(x1=x1^2,x2=x2^2,x3=x3^2,x4=x4^2)

def degn(f,n,b=b):
    g=f.taylor(b,0,n)
    L=g.coefficients(b)
    for i in range(len(L)):
        if L[i][1]==n:
            return L[i][0]
    return 0

def deg0(f):
    if f==0: 
        return 0
    return pretty(degn(f(t1=b*t1,t2=b*t2,t3=b*t3,t4=b*t4,m=b*m),0))


def fixed_locus(n):
    return list(pP4ds.with_num_boxes(n))
    
    
def sqrtK(w):
    K=x1*x2*x3*x4
    return sqrt(specialize(substitute(K,w)))

def determinant(f):
    if f in ZZ:
        return 1
    else:
        return prod((mon)^coeff for coeff, mon in f)

def Tvir(p,w=(x1,x2,x3,x4)):
    Q = specialize(character(p,w))
    Q_inv =dual(Q)
    K=sqrtK(w)
    P=(1-x1)*(1-x2)*(1-x3)*(1-x4)
    P=specialize(P)
    P=substitute(P,w)
    T = K*(1-(1-P*Q)*(1-dual(P*Q)))/P
    return T
    #return T.numerator()*dual(T.denominator())

def sqrtTvir(p,w=(x1,x2,x3,x4)):
    Q =specialize(character(p,w))
    Q_inv = dual(Q)
    P=(1-x1)*(1-x2)*(1-x3)*(1-x4)
    P=specialize(P)
    P=substitute(P,w)
    P0=substitute(specialize(dual((1-x1)*(1-x2)*(1-x3))),w)
    K=sqrtK(w)
    sqrtT = K*(Q-P0*Q*Q_inv)
    return sqrtT

def sqrtmono(f):
    numer, denom = 1,1
    for exp, coeff in f.dict().items():
        items=deque(exp)
        items.rotate(len(coho_vars))
        term = KT_ring({tuple(list(items)):1})
        numer *= term ** (coeff)
    return numer

def NekBracket(f):
    if f.parent()==ZZ:
        return 0
    R = f.parent()
    numer, denom = 1,1
    for exp, coeff in f.dict().items():
        if exp.is_constant():
            term = R.zero()
        else:
            items=deque(exp)
            items.rotate(len(coho_vars))
            term = KT_ring({tuple(list(items)):1})*(1-KT_ring({exp.emul(-1):1}))
            term = term
        if coeff > 0:
            numer *= term ** (coeff)
        elif coeff < 0:
            denom *= term ** (-coeff)
    return R.fraction_field()(numer) / denom

def Nek(n,w=(x1,x2,x3,x4)):
    total=0
    for SP in fixed_locus(n):
        add=SP.sign()*\
        NekBracket(dual(specialize(character(SP,w))*y^(-1))-sqrtTvir(SP,w))*(sqrtK(w)-1)^n
        add=add
        total+=add
    return total

def F(v=(x1,x2,x3,x4)):
    return specialize(substitute((x1^-1+x2^-1+x3^-1+x4^-1-x1-x2-x3-x4)/(1-x1)/(1-x2)/(1-x3)/(1-x4)*NekBracket(y)/(1-b*w)/(1-b*w^-1),v))


def F_list(w):
    return sum(F(w[i]) for i in range(8))


def sqrt_to_original(f):
    R = f.parent()
    term=0
    for exp, coeff in f.dict().items():
        newexp=()
        for i in range(len(coho_vars)):
            newexp+=(exp[i]+exp[i+len(coho_vars)]//2,)
        for i in range(len(coho_vars)):
            newexp+=(exp[len(coho_vars)+i]%2,)
        term +=R({newexp:coeff})
    return term
    
def cut(f):
    return sqrt_to_original(f.numerator())/sqrt_to_original(f.denominator())
    
def substitute(f,w):
    return f(x1=w[0],x2=w[1],x3=w[2],x4=w[3])

def Nek_list(n,w,sgn):
    total=0
    for i in range(8):
        add=sgn[i]*(Nek(n,w[i]))
        total+=add
    return total


def writeline(file,s):
    g = open(file, "a")
    g.write(s+"\n")
    g.close()
    
v=[(x1,x2,x3,x4),
   (x1,1/x2,x3/x2,x4/x2),
   (x1,1/x3,x2/x3,x4/x3),
   (x1,1/x4,x2/x4,x3/x4),
   (1/x1,x2,x3,x4),
   (1/x1,1/x2,x3/x2,x4/x2),
   (1/x1,1/x3,x2/x3,x4/x3),
   (1/x1,1/x4,x2/x4,x3/x4),
  ]

total=()

def genbin(n, bs=()):
    global total
    if len(bs) == n:
        total=total+(bs, )
    else:
        genbin(n, bs + (1,))
        genbin(n, bs + (-1,))

genbin(8,(1,))

In [11]:
pretty(Nek_list(2,v,total[4]))

208/15*w^2*(y - 1)^2/y^2

In [5]:
n=1
for i in range(128):
    a=Nek_list(n,v,total[i])(x1=1,x2=1,x3=1,u1=1,u2=1,u3=1,u4=1)(x4=1)*y^n/(1-y)^n/w^n
    print(a)

8
6
6
4
6
4
4
2
6
4
4
2
4
2
2
0
6
4
4
2
4
2
2
0
4
2
2
0
2
0


KeyboardInterrupt: 

In [12]:
n=3
for i in range(128):
    a=Nek_list(n,v,total[i])*y^n/(y-1)^n/w^n
    print(str(a)+', '+str(i))

-6024/175, 0
-152/5, 1
-12752/525, 2
-304/15, 3
-12752/525, 4
-304/15, 5
-7432/525, 6
-152/15, 7
-12752/525, 8
-304/15, 9
-7432/525, 10
-152/15, 11
-7432/525, 12
-152/15, 13
-704/175, 14


KeyboardInterrupt: 